In [37]:
from bs4 import BeautifulSoup
from selenium  import webdriver
import time
from selenium.common.exceptions import WebDriverException
import json
import pandas as pd
import re

#### For Testing
i='SOST'
URL= "https://www.genecards.org/cgi-bin/carddisp.pl?gene="+i+"&keywords="+i
driver = webdriver.Chrome("C:/driver/chromedriver.exe")
driver.get(URL)
time.sleep(4)
soup = BeautifulSoup(driver.page_source)
time.sleep(1)

In [38]:
def omim_id(soup):
    gene_ids=soup.find('section',attrs={'id':'aliases_descriptions'}).find('div',attrs={
        'class':'gc-subsection-inner-wrap'}).findAll('li')

    omim=[i.text for i in gene_ids if 'OMIM' in i.text] #results in a list like so - OMIM: omim_id
    omim_no=re.findall('[0-9]+', omim[0])[0] #results in a list, so extracting the first (and only) content of the list 
    return(omim_no)

In [39]:
def gene_name(soup):
    gene_alias=soup.find('span',attrs={'id':'aliasMainName'}).text #Basically the protein name
    return(gene_alias)

In [40]:
def genomic_location(soup):
    genomic_location=""
    m=soup.find('section',attrs={'id':'genomic_location'}).find(
        'div',attrs={'class':'col-xs-12 col-sm-4'}).findAll(['dt','dd'])
    for i in range(0,2) :
        genomic_location=genomic_location+(m[i].text) #adding the first two elements of the findall to get the genomic location 
    return(genomic_location)

In [41]:
def cytogenic_location(soup): #cytogenic location by HGNC
    cyt_location=soup.find('section',attrs={'id':'genomic_location'}).find(
            'dl',attrs={'class':'dl-horizontal gc-dl-9'}).find('li').text
    return(cyt_location)

In [42]:
def localization(soup):
    loc=json.loads(soup.find('section',attrs={'id':'localization'}).find(
        'div',attrs={'data-ng-controller':'goTermController'}).find(
        'div').get('data-json'))[0]['Term']
    return(loc)

In [43]:
def molecular_function(soup):
    mol_function=json.loads(soup.find('section',attrs={'id':'function'}).find(
            'div',attrs={'data-ng-controller':'goTermController'}).find(
            'div').get('data-json'))
    m_f=""
    for i in mol_function:
        m_f=m_f+i['Term']+", "
    return(m_f)

In [44]:
#Protein and gene interactions (data not present on genecards)

In [45]:
def pathways(soup): #only selecting the first pathway from genecards.org
    
    pathways=json.loads(soup.find('section',attrs={'id':'pathways_interactions'}).find(
                    'div',attrs={'id':'pathwayBySuperPathControllerComponent'}).find(
                    'div').get('data-json'))
    m_f=""
    for i in pathways:
        m_f=m_f+i['Name']+", "
    return(m_f)
    return(pathways)

In [46]:
def about_gene(soup):
    s=(soup.find('section',attrs={'id':'summaries'}).findAll(
        'div',attrs={'class':'gc-subsection'})[2].find('li').text).replace('\n',"").split('.')
    return(s[0])

In [47]:
def drugs(soup):
    try:#Some genes do not have drug data present
        drugs=json.loads(soup.find('section',attrs={'id':'drugs_compounds'}).find(
                    'div',attrs={'data-ng-controller':'UnifiedDrugController'}).find(
                    'div').get('data-json'))
        m_f=""
        for i in drugs:
            m_f=m_f+i['Name']+", "
        return(m_f)
    except:
        return("")

In [48]:
def interactions_regulators(soup):
    try:
        ir=soup.find('section',attrs={'id':'pathways_interactions'}).find(
            'dl',attrs={'class':'dl-horizontal dl-horizontal-unaligned'}).text.replace(
            ":\n\n\n\n",":").replace("\n\n\n\n"," ,").replace('\n'," ")
        return(ir)
    except:
        return("")

In [49]:
def diseases(soup):
    try:
        d=[]
        disorders=json.loads(soup.find('section',attrs={'id':'diseases'}).find(
                        'div',attrs={'data-ng-controller':'UnifiedDisorderTableController'}).find(
                        'div').get('data-json'))
        for i in disorders:
            d.append(i['Disorder']['Text'])
        return(d)
    except:
        return("")

Will store the gene names in a list, will iterate through the list to scrape info about genes and get the final dataframe for all the required genes

In [50]:
gene_list=['SOST','TNFRSF11B','TNFRSF11A','TNFSF11','GPC6','DAAM1','DAAM2','SLC2A1']

In [55]:
final_df=pd.DataFrame()
omim_ids=[]
for i in gene_list:
    URL= "https://www.genecards.org/cgi-bin/carddisp.pl?gene="+i+"&keywords="+i
    driver = webdriver.Chrome("C:/driver/chromedriver.exe")
    driver.get(URL)
    time.sleep(4)
    soup = BeautifulSoup(driver.page_source)
    time.sleep(1)
    
    #Extracting the OMIM id
    omim_ids.append(omim_id(soup))
    
    #Extracting the gene name
    gene_name1=gene_name(soup)
    time.sleep(1)

    genomic_location1=genomic_location(soup)
    
    cytogenic_location1=cytogenic_location(soup)
    time.sleep(1)

    localization1=localization(soup)
    molecular_function1=molecular_function(soup)
    protein_interactions1=""
    pathways1=pathways(soup)
    about_gene1=about_gene(soup)
    drugs1=drugs(soup)
    interactions_regulators1=interactions_regulators(soup)
    diseases1=diseases(soup)
    function=""

    final={i:(gene_name1,genomic_location1,cytogenic_location1,localization1,molecular_function1,
              protein_interactions1,pathways1,about_gene1,drugs1,interactions_regulators1,diseases1,function)}
    final_df=pd.concat([final_df,pd.DataFrame(final)],axis=1)

In [56]:
ind=['Gene Product','Cytogenic Location','Genomic Coordinates','Localization','Molecular Function(GO)',
    'Protein/Gene Interactions','Pathways Involved','About the gene','Drug Interactions','Regulators',
    'Associated Diseases','Function']
final_df.index=ind
final_df

,SOST,TNFRSF11B,TNFRSF11A,TNFSF11,GPC6,DAAM1,DAAM2,SLC2A1
Gene Product,Sclerostin,TNF Receptor Superfamily Member 11b,TNF Receptor Superfamily Member 11a,TNF Superfamily Member 11,Glypican 6,Dishevelled Associated Activator Of Morphogene...,Dishevelled Associated Activator Of Morphogene...,Solute Carrier Family 2 Member 1
Cytogenic Location,"chr17:43,753,738-43,758,791(GRCh38/hg38)","chr8:118,923,557-118,951,885(GRCh38/hg38)","chr18:62,325,287-62,391,288(GRCh38/hg38)","chr13:42,562,736-42,608,013(GRCh38/hg38)","chr13:93,226,807-94,408,020(GRCh38/hg38)","chr14:59,188,646-59,371,405(GRCh38/hg38)","chr6:39,792,298-39,904,877(GRCh38/hg38)","chr1:42,925,353-42,958,868(GRCh38/hg38)"
Genomic Coordinates,17q21.31 by HGNC,8q24.12 by HGNC,18q21.33 by HGNC,13q14 by HGNC,13q31.3-q32.1 by HGNC,14q23.1 by HGNC,6p21.2 by HGNC,1p34.2 by HGNC
Localization,extracellular region,extracellular region,cytosol,extracellular region,extracellular region,stress fiber,extracellular exosome,Golgi membrane
Molecular Function(GO),"protein binding, transcription factor binding,...","cytokine activity, protein binding, signaling ...","transmembrane signaling receptor activity, tum...","cytokine activity, tumor necrosis factor recep...","protein binding, coreceptor activity involved ...","actin binding, protein binding, Rho GTPase bin...","actin binding, Rho GTPase binding,","long-chain fatty acid transporter activity, gl..."
Protein/Gene Interactions,,,,,,,,
Pathways Involved,"Signaling by Wnt, Parathyroid hormone synthesi...",TNF Superfamily - Human Ligand-Receptor Intera...,"Akt Signaling, RANK Signaling in Osteoclasts, ...","PEDF Induced Signaling, ERK Signaling, RANK Si...","Apoptotic Pathways in Synovial Fibroblasts, Ch...","Wnt signaling pathway, Regulation of activated...","Wnt signaling pathway, WNT Signaling,","Transport of glucose and other sugars, bile sa..."
About the gene,Negative regulator of bone growth that acts th...,Acts as decoy receptor for TNFSF11/RANKL and t...,Receptor for TNFSF11/RANKL/TRANCE/OPGL; essent...,Cytokine that binds to TNFRSF11B/OPG and to TN...,Cell surface proteoglycan that bears heparan s...,"Binds to disheveled (Dvl) and Rho, and mediate...",HGNC(18143),"Facilitative glucose transporter, which is res..."
Drug Interactions,"Romosozumab,","Anastrozole, Letrozole, Calcium,","Denosumab, Acetaminophen, Aspirin, Diclofenac,...","Lenalidomide, Denosumab, Anastrozole, Letrozol...",,"Guanosine triphosphate,",,"D-glucose, Vitamin C, Butabarbital, carboxymet..."
Regulators,"Inactivates:WNT3A ,Is activated by:BMPR1A ,","Is activated by:TNFSF11 ,Is inactivated by:CR...","Is activated by:TNFSF11 TRAF6 ,","Activates:TNFRSF11A TNFRSF11B ,Is activated b...",,"Activates:RAC1 RHOA ,Is activated by:DVL1P1 D...",,"Is activated by:MYC SP1 AKT1 ,Is inactivated ..."


In [53]:
final_df.to_csv("Osteoporosis_genes.csv")

# Downloading the gene variant data from OMIM

In [ ]:
for i in omim_ids:
    URL="https://omim.org/allelicVariants/"+i
    driver = webdriver.Chrome("C:/driver/chromedriver.exe")
    driver.get(URL)
    time.sleep(4)
    soup = BeautifulSoup(driver.page_source)
    time.sleep(1)
    
    
    

In [57]:
URL="https://omim.org/allelicVariants/"+i
driver = webdriver.Chrome("C:/driver/chromedriver.exe")
driver.get(URL)
time.sleep(4)
soup = BeautifulSoup(driver.page_source)
time.sleep(1)

['605740',
 '602643',
 '603499',
 '602642',
 '604404',
 '606626',
 '606627',
 '138140']